In [ ]:
import cv2

video_path = "C:\\Users\\chand\\Trad._CV\\KOHLI_COVER_DRIVE.mp4"
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print(f"Error: Could not open video file at {video_path}")
    exit()

frame_count = 0  

print("Starting frame processing...")

while cap.isOpened():
    ret, frame = cap.read()
    
    if not ret:
        print("End of video or error reading frame.")
        break  

    frame_count += 1
    print(f"Processing frame {frame_count}")

cap.release()
print(f"Total frames processed: {frame_count}")


In [ ]:
import cv2
import matplotlib.pyplot as plt

video_path = "C:\\Users\\chand\\Trad._CV\\KOHLI_COVER_DRIVE.mp4"
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print(f"Error: Could not open video file at {video_path}")
    exit()

ret, frame = cap.read()

if not ret:
    print("Error: Could not read the first frame.")
    cap.release()
    exit()

x_start, x_end = 100, 300
y_start, y_end = 50, 200
roi = frame[y_start:y_end, x_start:x_end] 

roi_rgb = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)

plt.imshow(roi_rgb)
plt.axis("off")  
plt.show()

cap.release()


In [ ]:
import cv2
import numpy as np
import imutils
import regex as re
import matplotlib.pyplot as plt
import os

os.makedirs("Frame", exist_ok=True)
os.makedirs("Frame_b", exist_ok=True)

video_path = "C:\\Users\\chand\\Trad._CV\\KOHLI_COVER_DRIVE.mp4"
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print(f"Error: Could not open video file at {video_path}")
    exit()

frame_count = 0  

print("Processing video frames...")


ret, first_frame = cap.read()
if not ret:
    print("Error: Could not read the first frame.")
    cap.release()
    exit()

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("End of video or error reading frame.")
        break  

    roi = frame.copy()  # Keeping the frame untouched
    plt.imshow(cv2.cvtColor(roi, cv2.COLOR_BGR2RGB))
    plt.show()

    if cv2.waitKey(50) & 0xFF == ord("q"):
        break  

    cv2.imwrite(f"Frame/{frame_count}.png", roi)
    frame_count += 1

cap.release()
cv2.destroyAllWindows()

print(f"Total frames extracted: {frame_count}")


frames = sorted(os.listdir("Frame/"), key=lambda f: int(re.sub("\\D", "", f)))

processed_images = []
trajectory_points = []
radius_list = []
bounding_boxes = []

for frame_filename in frames:
    frame_path = os.path.join("Frame/", frame_filename)
    img = cv2.imread(frame_path)
    if img is None:
        print(f"Warning: Could not read {frame_filename}, skipping...")
        continue

    processed_img = img.copy()
    hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

   
    lower_red1, upper_red1 = np.array([0, 100, 0]), np.array([10, 255, 255])
    lower_red2, upper_red2 = np.array([160, 100, 20]), np.array([180, 255, 255])

    mask1 = cv2.inRange(hsv_img, lower_red1, upper_red1)
    mask2 = cv2.inRange(hsv_img, lower_red2, upper_red2)
    combined_mask = mask1 + mask2

 
    processed_img = cv2.bitwise_and(processed_img, processed_img, mask=combined_mask)


    contours, _ = cv2.findContours(combined_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    detected_blobs = []
    analyzed_img = img.copy()

    for contour in contours:
        blob_area = cv2.contourArea(contour)
        blob_perimeter = cv2.arcLength(contour, True)

        if blob_perimeter == 0:
            continue  

        blob_circularity = (4 * np.pi * blob_area) / (blob_perimeter**2)

     
        min_circularity = 0.2
        min_area = 35

     
        (x, y), radius = cv2.minEnclosingCircle(contour)
        center = (int(x), int(y))
        radius = int(radius)

        if blob_circularity > min_circularity and blob_area > min_area:
            detected_blobs.append([center, radius, blob_area, blob_circularity])

    if detected_blobs:
        largest_blob = max(detected_blobs, key=lambda b: b[2] * b[3])
        best_center, best_radius, best_area, best_circularity = largest_blob

        if best_circularity < 0.4 and len(detected_blobs) > 1:
            other_blobs = [b for b in detected_blobs if b != largest_blob]
            best_blob = max(other_blobs, key=lambda b: b[2] * b[3])
            best_center, best_radius, best_area, best_circularity = best_blob

        box_width, box_height = 4 * best_radius, 4 * best_radius
        color = (255, 0, 0)  # Blue for circle

        x, y = best_center
        cv2.circle(analyzed_img, best_center, best_radius, color, 2)
        cv2.rectangle(analyzed_img, (x - box_width // 2, y - box_height // 2),
                      (x + box_width // 2, y + box_height // 2), (0, 0, 255), 3)

        trajectory_points.append(best_center)
        radius_list.append(best_radius)
        bounding_boxes.append([box_width, box_height])
        processed_images.append(analyzed_img)

output_dir = "Frame_b/"
for i, processed_frame in enumerate(processed_images):
    cv2.imwrite(os.path.join(output_dir, f"processed_frame_{i}.png"), processed_frame)

print("Processed frames saved.")

frames = sorted(os.listdir("Frame_b/"), key=lambda f: int(re.sub("\\D", "", f)))
frame_array = []

for frame_name in frames:
    img = cv2.imread(os.path.join("Frame_b/", frame_name))
    if img is None:
        continue

    height, width, layers = img.shape
    size = (width, height)
    frame_array.append(img)

output_video_path = "Cric_with_bounding_boxes.mp4"
video_out = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*"DIVX"), 20, size)

for frame in frame_array:
    video_out.write(frame)

video_out.release()

print(f"Output video saved as {output_video_path}")
